# Kaggle Time serie tutorial applied to Apnea data

(Inspired by https://www.kaggle.com/code/konradb/ts-0-the-basics?scriptVersionId=106894931)

In [ ]:
import pandas as pd
import numpy as np
from random import gauss
from pandas.plotting import autocorrelation_plot
import warnings
import itertools
from random import random
from os import listdir, path
from os.path import isfile, join
from tqdm.notebook import tqdm

import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller

import matplotlib.pyplot as plt
#plt.style.use('fivethirtyeight')
# import matplotlib as mpl
import seaborn as sns

# v0.0.3
from pyapnea.oscar.oscar_loader import load_session
from pyapnea.oscar.oscar_getter import event_data_to_dataframe, get_channel_from_code
from pyapnea.oscar.oscar_constants import CHANNELS, ChannelID

In [ ]:
# adjust the parameters for displayed figures
plt.rcParams.update({'figure.figsize': (30, 10)})

In [ ]:
filename_to_load = '../data/raw/ResMed_23192565579/Events/62202198.001'
oscar_session_data = load_session(filename_to_load)
df = event_data_to_dataframe(oscar_session_data, [ChannelID.CPAP_FlowRate.value])
df[['time_utc', 'FlowRate']].set_index('time_utc').plot()

In [ ]:
# decomposition
# Period : 4s (approximately) of inspiration-expiration => 4000 ms / 40 ms (freq = 25Hz) = 100
# Period : 1 min  => 60000 ms / 40 ms (freq = 25Hz) = 1500
period = 100
df_nozero = df.copy()
df_nozero['FlowRate'] = df_nozero['FlowRate'] + 50
decomposition = sm.tsa.seasonal_decompose(df_nozero["FlowRate"].iloc[0:5000], period=period, model='additive')
figure = decomposition.plot()
plt.show()

In [ ]:
# LAGS ??
plot_acf(df['FlowRate'], lags = period)

In [ ]:
df['FlowRate'].plot.hist(bins=50, alpha=0.5)

In [ ]:
X = df['FlowRate'].values
split =  int(len(X) / 2)
X1, X2 = X[0:split], X[split:]
mean0, mean1, mean2 = X.mean(), X1.mean(), X2.mean()
var0, var1, var2 = X.var(), X1.var(), X2.var()
print('mean:')
print('all: %.2f, chunk1: %.2f vs chunk2: %.2f' % (mean0, mean1, mean2))
print('variance:')
print('all: %.2f, chunk1: %.2f vs chunk2: %.2f' % (var0, var1, var2))

NOTE : Not continuing this notebook. The aims is signal forecasting and it's not interesting for apena event detection (anomaly detection)